# Horoscope generation using Temporal Convolution Networks

The goal of this notebook is to implement a generator of horoscope based on neural networks. 

More specifically, the architecture used is a Temporal Convolution Network based on the research paper ["An Empirical Evaluation of Generic Convolutional and Recurrent Networks for Sequence Modeling"](https://arxiv.org/abs/1803.01271). This architecture is fully convolutional and can therefore take arbitrary length sequence as inputs. The main idea of the authors of this paper is to increase the perceptive field of each successive layer using [dilated convolution](https://github.com/vdumoulin/conv_arithmetic). 

The bulk of the code of the TCN has been taken from [the implementation](https://github.com/locuslab/TCN) linked in the article.

The data used here are all the horoscopes published [beliefnet.com](beliefnet.com) for the year 2011. 

The network takes a sequence of `window_size` words as input and outputs a sequence of `window_size` words. The target that we use for training is a slice of the horoscope corresponding to the input slid on step to the right, effectively asking the network what should the next word be. 

## Imports

In [1]:
import random
import math
import itertools

import numpy as np
import pandas as pd

from tqdm import tnrange, tqdm_notebook
from tqdm import tqdm

import torch
import torch.nn            as nn
import torch.nn.functional as F
import torch.optim         as optim
from torch.nn.utils import weight_norm
from torch.autograd import Variable

## Training parameters

In [2]:
cuda          = True
file_path     = '../data/horoscope_2011.csv'
window_size   = 50
batch_size    = 256
print_every   = 200
test_seq_size = 100
epochs        = 30

## Data loading code

In [3]:
def load_data(path, window_size):
    df          = pd.read_csv(path)
    split_texts = [t.lower().split() for t in df.TEXT] 
    text        = list(itertools.chain.from_iterable(split_texts))
    words       = set(text)
    n_words     = len(words)
    idx_to_word = dict(enumerate(words))
    word_to_idx = {word : idx for idx, word in idx_to_word.items()}
    data        = [(text[i : i + window_size], text[i + 1 : i + window_size + 1])
                    for i in range(len(text) - window_size - 1)]

    return n_words, idx_to_word, word_to_idx, data

In [4]:
def encode_seq(seq, word_to_id):
    return [word_to_id[w] for w in seq]

In [5]:
def data_to_tensor(data, word_to_id, n_char):
    input_tensor = torch.LongTensor([encode_seq(input_seq, word_to_id)
                                      for input_seq, _ in data])
    target_tensor = torch.LongTensor([encode_seq(target_seq, word_to_id)
                                       for _, target_seq in data])
    
    return input_tensor, target_tensor

In [6]:
def batch_generator(data, batch_size, n_char, word_to_id, shuffle = True):
    if shuffle:
        data = random.sample(data, len(data))
    
    return (data_to_tensor(data[i : i + batch_size], word_to_id, n_char) 
                 for i in range(0, len(data), batch_size))

## Model visualization code

The model evaluation consists in asking it to generate a long sequence of character. We randomly select an input as a base for our generation and create a new sequence character by character using the model.

In [7]:
def test_model(tcn, final_sequence_size, window_size, n_words, 
               id_to_word, word_to_id, data):
    seq = list(random.choice(data)[0])
    while len(seq) < final_sequence_size:
        # As the sequence is able to take variable length inputs, it could be 
        # interesting to not limit ourselves on inputs of window_size.
        encoded_input = encode_seq(seq[-window_size:], word_to_id)
        input_tensor  = torch.LongTensor([encoded_input])
        X             = Variable(input_tensor)
        X             = X.cuda() if cuda else X 
        y_pred        = tcn(X)
        # It is important to take the maximum on the dim -2 as each channel of 
        # the output will correspond to the score associated to a character.
        word_pred_id  = y_pred.cpu().max(dim = 2)[1][:,-1].data[0]
#         word_pred_id  = y_pred[0].max(dim = -2)[1][-1].cpu().data[0]
        word_pred     = id_to_word[word_pred_id]
        seq.append(word_pred)
    
    return ' '.join(seq)

The following function generates an horoscope starting from a `base` supplied by the caller. 

In [8]:
def genererate_long_sequence(tcn, final_sequence_size, n_wordsr, id_to_word, word_to_id, base):
    seq = base.split()

    while len(seq) < final_sequence_size:
        # In this case we do not limit the size of the input to window_size.
        encoded_input = encode_seq(seq, word_to_id)
        input_tensor  = torch.LongTensor([encoded_input])
        X             = Variable(input_tensor)
        X             = X.cuda() if cuda else X 
        y_pred        = tcn(X)
        # It is important to take the maximum on the dim -2 as each channel of 
        # the output will correspond to the score associated to a character.
        word_pred_id  = y_pred[0].max(dim = -2)[1][-1].cpu().data[0]
        word_pred     = id_to_word[word_pred_id]
        seq.append(word_pred)
        
    return ' '.join(seq)

## Model definition

The `TransposeLayer` is just a simple transposition that reformats the output of the embedding layer into the correct format for the convolution layers.

In [9]:
class TransposeLayer(nn.Module):
    def forward(self, x):
        return x.transpose(-2, -1)

The `Chomp1d` module is used to remove the extra values at the end of the sequence by the padding of a convolution. As the TCN architecture uses dilated convolution, the padding have to be increased in order to be able to generate a long enough output. We have to remove the extra values so that the last value of our output is the result of a dilated convolution whose rightmost value was the last value of the input sequence. 

In [10]:
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size
        
    def forward(self, x):
        # As x can be stored on the GPU, if we use it to build a new tensor,
        # we have to ensure that our new value is stored contiguously.
        return x[:, :, :-self.chomp_size].contiguous()

The `TemporalBlock` module is a residual block containing two weight normalized dilated convolutions with ReLU activations and dropout2d (we drop whole channel at once). The residual connection may contain a 1x1 convolution if it is necessary to transform the input to the correct number of channels. 

In [11]:
class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout = 0.2):
        super(TemporalBlock, self).__init__()
        conv_params = {
            'kernel_size' : kernel_size,
            'stride'      : stride,
            'padding'     : padding,
            'dilation'    : dilation
        }
        self.conv1    = weight_norm(nn.Conv1d(n_inputs, n_outputs, **conv_params))
        self.chomp1   = Chomp1d(padding)
        self.relu1    = nn.ReLU()
        self.dropout1 = nn.Dropout2d(dropout)
        self.conv2    = weight_norm(nn.Conv1d(n_outputs, n_outputs, **conv_params))
        self.chomp2   = Chomp1d(padding)
        self.relu2    = nn.ReLU()
        self.dropout2 = nn.Dropout2d(dropout)
        self.net      = nn.Sequential(
            self.conv1, 
            self.chomp1,
            self.relu1,
            self.dropout1,
            self.conv2,
            self.chomp2,
            self.relu2,
            self.dropout2
        )
        # If the number of input channels is equal to the number of output channel then
        # no transformation is required.
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu       = nn.ReLU()
        self.init_weights()
        
    def forward(self, x):
        # Convolutional branch of the residual block
        out = self.net(x)
        # Residual branch of the residual block
        res = x if self.downsample is None else self.downsample(x)

        return self.relu(out + res)
    
    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

The Temporal Convolution Network is a sequence of Temporal Blocks whose dilation is doubled at each step. If enough blocks are uses, this definition allows the network to used information for arbitrarily far away in the past to generate its prediction. 

In [12]:
class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, dim_emb, kernel_size, dropout):
        super(TemporalConvNet, self).__init__()
        layers     = []
        num_levels = len(num_channels)
        
        for i in range(num_levels):
            # The dilation is doubled at each layer to allow an exponential growth of 
            # the receptive field size.   
            dilation_size = 2 ** i
            in_channels   = dim_emb if i == 0 else num_channels[i - 1]
            out_channels  = num_channels[i]
            layers.append(
                TemporalBlock(
                    in_channels,
                    out_channels,
                    kernel_size,
                    stride   = 1,
                    dilation = dilation_size,
                    padding  = (kernel_size - 1) * dilation_size,
                    dropout  = dropout
                )
            )
        
        self.network = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.network(x)

In [13]:
class LanguageModel(nn.Module):
    def __init__(self, n_inputs, n_outputs, num_channels, dim_emb = 50, kernel_size = 2, 
                 emb_dropout = 0.1, dropout = 0.2):
        super(LanguageModel, self).__init__()
        self.embedding = nn.Embedding(n_inputs, dim_emb)
        self.tcn = TemporalConvNet(dim_emb, num_channels, dim_emb, kernel_size, dropout)
        self.decoder = nn.Linear(num_channels[-1], n_outputs)
        self.drop = nn.Dropout(emb_dropout)
        self.init_weights()
        
    def init_weights(self):
        self.embedding.weight.data.normal_(0, 0.01)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.normal_(0, 0.01)
        
    def forward(self, input):
        emb = self.drop(self.embedding(input))
        emb = emb.transpose(-2, -1)
        y   = self.tcn(emb)
        y   = y.transpose(-2, -1) 
        y   = self.decoder(y)
        
        return y

## Training

In [14]:
n_words, id_to_word, word_to_id, data = load_data(file_path, window_size)
# num_channels                          = [512] * 6 + [n_words]
num_channels                          = [512] * 6
# tcn                                   = TemporalConvNet(n_words, num_channels)
tcn                                   = LanguageModel(n_words, n_words, num_channels) 
tcn                                   = tcn.cuda() if cuda else tcn
# We view the problem as a classification task in which the network tries
# to predict what class the following character should be.   
criterion                             = nn.CrossEntropyLoss()
# We use an Adam optimizer with the default learning rate of 1e-3.
optimizer                             = optim.Adam(tcn.parameters())
# tcn

In [15]:
params = filter(lambda p: p.requires_grad, tcn.parameters())
sum([np.prod(p.size()) for p in params])

13234883

In [16]:
batch_per_epoch  = math.ceil(len(data) / batch_size)
loss_update_rate = 3

In [17]:
for epoch in tnrange(epochs, desc = 'epochs'):
    loss_pbar    = 0 
    running_loss = 0
    generator    = batch_generator(data, batch_size, n_words, word_to_id)

    with tqdm_notebook(
        enumerate(generator), 
        desc = 'batches', 
        total = batch_per_epoch, 
        unit = 'batch '
    ) as pbar:

        for i, (X, y) in pbar:
            X = Variable(X)
            y = Variable(y)
            X = X.cuda() if cuda else X
            y = y.cuda() if cuda else y
            optimizer.zero_grad()
            y_pred = tcn(X)
            reshaped_y      = y.view(-1)
            reshaped_y_pred = y_pred.view(-1, n_words) 
            loss   = criterion(reshaped_y_pred, reshaped_y)
            loss.backward()
            optimizer.step()

            loss_value    = loss.cpu().data[0]
            running_loss += loss_value
            loss_pbar    += loss_value

            if i % loss_update_rate == loss_update_rate - 1:
                pbar.set_postfix(loss = loss_pbar / loss_update_rate)
                loss_pbar = 0

            if i % print_every == print_every - 1:
                test_result = test_model(tcn, test_seq_size, window_size, n_words, 
                                         id_to_word, word_to_id, data)
                tqdm.write(f'Batch: {i + 1 : 6}, '
                           f'loss: {running_loss / print_every : .4f}\n'
                           f'{test_result}\n')
                running_loss = 0

HBox(children=(IntProgress(value=0, description='epochs', max=30), HTML(value='')))

HBox(children=(IntProgress(value=0, description='batches', max=1260), HTML(value='')))

Batch:    200, loss:  6.1580
you are not responsible for anyone else's emotions. don't intrude into someone else's life unless you have been specifically invited. your ruling planet mercury engages in a mental war of survival with relentless pluto today. you may feel as if you're in over your head, but mercury is clever enough to be as you may be be able to be a feelings of your current house of moon is to your current way of your sign to be able to be a moon and your moon house of moon is your moon and your moon house of work is a

Batch:    400, loss:  4.8890
the day because you might become quite busy later in the afternoon. but instead of just daydreaming about how to make more money, it would be wise to consider ways to reduce your spending. asking a friend for assistance could make a significant difference in what you accomplish now, for your feelings is a good idea to make your feelings and a few days. you may be able to make your feelings and a good strategy to be a bit of two 

HBox(children=(IntProgress(value=0, description='batches', max=1260), HTML(value='')))

Batch:    200, loss:  2.8036
a few loose ends dangling now. delay declaring your final decision until the ambiguity begins to fade. unraveling your romantic feelings may be a significant part of your day, yet it's difficult to know where your heart will lead you now. you could grow frustrated if you try to force clarity. don't let your frustration is so firm that you cannot see something. nevertheless, you may not be able to address your current optimism, but you might be struggling to sustain your energy to yourself. you may be extremely successful in the day than you realize. you may be feeling

Batch:    400, loss:  2.6637
where they may. you are extra creative today as mars, now in your 5th house of self-expression, is agitated by imaginative neptune. however, your production can be delayed or even blocked by your lack of a concrete plan. nevertheless, you could feel as if you're wasting time managing the unexpected interruptions, but it's wise to discuss potential to business and 

...

Batch:    200, loss:  0.7359
be overwhelmed by the mixed signals you are receiving today. one moment, all systems are go. but, then, just a few minutes later, everything is on hold. although you could act impulsively and try to fix what's wrong, this reaction probably would be a mistake. instead, think about which changes before you start off to the next level of intimacy. but you're not a stranger to the true moment of your own sense of self-confidence is enough to make the best of a new course of intimacy. it's helpful that you are unable to avoid the potential in such as

Batch:    400, loss:  0.7356
you have been working late into the night, take stock of what you have already accomplished and let others know what you have left to do. don't waste energy on self-judgment; just get some rest today and tomorrow you will feel refreshed and ready for anything. if you have been waiting for this weekend. it may be a good idea for everyone diligently toward making a bit of peace and quiet 

HBox(children=(IntProgress(value=0, description='batches', max=1260), HTML(value='')))

Batch:    200, loss:  0.7278
can't steer you right. instead of making circumstances worse, do what you should when you get lost. stop! don't go anywhere until you have regained your bearings. ask for directions, if necessary. then, continue on your way. you don't want to move very fast now and prefer that you finish one task through all of change. your need to money is probably more fulfilling. you have a lot to learn today and be prepared to work hard and do your job should begin to settle down later in the week. you're eager to describe your emotions with your friends, yet

Batch:    400, loss:  0.7280
you are confident that you can pull everything back together. actually, you might even enjoy all the action today. surprisingly, you could get what you want if you maintain a healthy perspective so you don't act too impulsively. slow down and turn on your charm; your witty ways may score you the extra points of time than you think. remember what you are thinking, but you may not see th

HBox(children=(IntProgress(value=0, description='batches', max=1260), HTML(value='')))

Batch:    200, loss:  0.7199
regain firm footing. even if you don't immediately see positive results from today's efforts, rest assured that your persistence will eventually have the effect you desire. you're tempted to act in an overly demanding manner today because you may believe that you deserve more than you have. unfortunately, the squeaky wheel doesn't always get the grease and you could receive a greater payoff by just being patient, instead. value-conscious venus in your 10th house of status forms a supportive sextile with calculating saturn, giving you and how you have fun for a while for the details of the situation into

Batch:    400, loss:  0.7201
heart. your ruling planet venus, now visiting your 10th house of career, provokes feelings of loneliness as she squares judgmental saturn. emotional stress that has been pressurizing for a while could suddenly explode and turn your professional life upside down. instead of worrying about the unknown, embrace the upcoming shift. 

In [19]:
# genererate_long_sequence(
#     tcn, 
#     5000, 
#     n_words, 
#     id_to_word, 
#     word_to_id, 
#     'your day will be bad but you should stay optimistic because'
# )